# Creating the Main Dataset

### This notebook takes in different Government data sources and joins it with the hospital data



In [1]:
# Import the dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter('ignore')

pd.set_option('display.max_columns', 50)

## Reading in the Hospital Dataset

The hospital dataset is obtained from the Australian Institute of Health and Welfare, specifically at this link -> https://www.aihw.gov.au/reports/hospitals/hospital-resources-2017-18-ahs/data

This dataset is highly useful as it shows the number of available beds at the hospital level. There is an additional column "Postcode" which was added by me manually - this is the power behind the dataset as we can then match a postcode with an SA2 (which has all the demographic data in - seen in the cells below).

In [2]:
hospitals = pd.read_excel("Data\Public_Hospitals_with_postcode.xlsx")
hospitals.head()

,State,Hospital name,Hospital name Revised,Establishment ID,Medicare provider no.,Local Hospital Network code,Local Hospital Network,Remoteness area (code),Remoteness area,Number of available beds,Peer group code,Peer group name,Supplied Establishments data,Supplied Morbidity data,Supplied Emergency department data,Supplied Elective Surgery Waiting times data,Supplied Non-admitted patient aggregate data,IHPA funding designation,Postcode
0,NSW,Tomaree Community,Tomaree Community Hospital,1180Q2250,0010000F,109,Hunter New England,1,Inner Regional,14.0,S2,Public acute group D,Yes,Yes,Yes,No,Yes,Block-funded hospital,2315
1,NSW,Auburn,Auburn Hospital,1174D2010,0010020A,104,Western Sydney,0,Major Cities,112.0,M,Public acute group B,Yes,Yes,Yes,Yes,Yes,Activity-based funded hospital,2144
2,NSW,Balmain,Balmain Hospital,1170A2010,0010030Y,102,Sydney,0,Major Cities,81.2,S2,Public acute group D,Yes,Yes,No,No,Yes,Activity-based funded hospital,2041
3,NSW,Bankstown/ Lidcombe,Bankstown Lidcombe Hospital,1171D2270,0010040X,103,South Western Sydney,0,Major Cities,390.0,L,Public acute group A,Yes,Yes,Yes,Yes,Yes,Activity-based funded hospital,2200
4,NSW,Shellharbour,Shellharbour Hospital,1173P2110,0010060T,108,Illawarra Shoalhaven,0,Major Cities,176.4,M,Public acute group B,Yes,Yes,Yes,Yes,Yes,Activity-based funded hospital,2528


In [3]:
hospitals.shape

(697, 19)

## Aggregate Hospital Beds to Postcode

Because multiple hospitals can belong to 1 suburb (postcode), the code below aims to aggregate the number of available hospital beds to it's postcode.

In [4]:
Unique_postcode = hospitals.groupby(["Postcode"]).agg({"Number of available beds": ['sum']})
Unique_postcode.reset_index(inplace=True)
Unique_postcode.columns = Unique_postcode.columns.droplevel(-1)
Unique_postcode.head()

,Postcode,Number of available beds
0,810,1243.0
1,829,66.0
2,850,82.0
3,860,43.0
4,870,319.0


## Map the postcode with an SA2

To obtain the features to help predict the number of hospital beds for a new area, a mapping between the postcode and SA2 is required, because the SA2 contains all the useful census data.

In [5]:
postcode_sa2 = pd.read_csv("Data\SA2_Postcode_Map.csv")
postcode_sa2.head()

,LOCALITY_ID,LOCALITY_NAME,LOCALITY_TYPE,POSTCODE,STATE,SA2_MAINCODE,SA2_NAME
0,ACT107,ACTON,GAZETTED LOCALITY,2601,ACT,801051049,Acton
1,ACT107,ANU,ALIAS LOCALITY,2601,ACT,801051049,Acton
2,ACT107,AUSTRALIAN NATIONAL UNIVERSITY,ALIAS LOCALITY,2601,ACT,801051049,Acton
3,ACT107,SPINNAKER ISLAND,ALIAS LOCALITY,2601,ACT,801051049,Acton
4,ACT107,SPRINGBANK ISLAND,ALIAS LOCALITY,2601,ACT,801051049,Acton


In [6]:
postcode_sa2 = postcode_sa2[["POSTCODE", "STATE", "SA2_MAINCODE","SA2_NAME"]]
postcode_sa2.drop_duplicates()

,POSTCODE,STATE,SA2_MAINCODE,SA2_NAME
0,2601,ACT,801051049,Acton
5,2601,ACT,801051123,Black Mountain
6,2602,ACT,801051050,Ainslie
8,2914,ACT,801041034,Amaroo
9,2614,ACT,801011001,Aranda
...,...,...,...,...
33713,6164,WA,507011163,Yangebup
33721,6218,WA,501021014,Harvey
33731,6372,WA,509031247,Kulin
33735,6571,WA,509021241,Moora


# Find the number of hospital beds by SA2

At this point it was discovered that 1 postcode can belong to multiiple SA2s. After a lot of consideration I decided to leave it this way and address any anomalies in the EDA Notebook.

In [7]:
hospital_sa2 = pd.merge(Unique_postcode, postcode_sa2, left_on="Postcode",
                       right_on="POSTCODE", how="inner")

hospital_sa2.drop_duplicates(inplace=True)

hospital_sa2.head(15)

,Postcode,Number of available beds,POSTCODE,STATE,SA2_MAINCODE,SA2_NAME
0,810,1243.0,810,NT,701021010,Alawa
1,810,1243.0,810,NT,701021013,Brinkin - Nakara
3,810,1243.0,810,NT,701021016,Coconut Grove
5,810,1243.0,810,NT,701021018,Jingili
7,810,1243.0,810,NT,701021021,Lyons (NT)
8,810,1243.0,810,NT,701021023,Millner
9,810,1243.0,810,NT,701021024,Moil
12,810,1243.0,810,NT,701021027,Tiwi
14,810,1243.0,810,NT,701021028,Wagaman
16,810,1243.0,810,NT,701021029,Wanguri


In [8]:
hospital_sa2.shape

(1444, 6)

In [9]:
hospital_beds_by_sa2 = hospital_sa2.groupby(["SA2_NAME", "SA2_MAINCODE"]).agg({"Number of available beds": ['sum']})


In [10]:
hospital_beds_by_sa2.reset_index(inplace=True)
hospital_beds_by_sa2.columns = hospital_beds_by_sa2.columns.droplevel(-1)
hospital_beds_by_sa2.head()


,SA2_NAME,SA2_MAINCODE,Number of available beds
0,ACT - South West,801111140,45.7
1,Acton - Upper Burnie,604011075,145.0
2,Adelaide,401011001,812.0
3,Adelaide Hills,401021003,17.0
4,Airlie - Whitsundays,312031359,33.0


In [11]:
hospital_beds_by_sa2.shape

(988, 3)

## Population Dataset

The Australian Bureau of Statistics provides population data with different cuts of the data, the source is publicly available here: https://www.abs.gov.au/statistics/people/population/national-state-and-territory-population/sep-2020#notes

For the purpose of this dataset, we will only be looking at the total number of any person within an SA2 area.

In [12]:
population = pd.read_csv("Data\Population_age_sex.csv")
population = population[(population["Sex"] == "Persons") & (population["AGE"] == "TT")]
population.head()

,MEASURE,Measure,SEX_ABS,Sex,AGE,Age,REGIONTYPE,Region Type,ASGS_2011,Region,FREQUENCY,Frequency,TIME,Time,Value,Flag Codes,Flags
5,ERP,Estimated Resident Population,3,Persons,TT,All ages,SA2,Statistical Area Level 2,314011386,Hills District,A,Annual,2016,2016,24226,NaN,NaN
20,ERP,Estimated Resident Population,3,Persons,TT,All ages,SA2,Statistical Area Level 2,214011376,Skye - Sandhurst,A,Annual,2016,2016,13664,NaN,NaN
24,ERP,Estimated Resident Population,3,Persons,TT,All ages,SA2,Statistical Area Level 2,401021006,Lobethal - Woodside,A,Annual,2016,2016,9101,NaN,NaN
38,ERP,Estimated Resident Population,3,Persons,TT,All ages,SA2,Statistical Area Level 2,212021294,Berwick - South,A,Annual,2016,2016,26874,NaN,NaN
48,ERP,Estimated Resident Population,3,Persons,TT,All ages,SA2,Statistical Area Level 2,308011191,Central Highlands - West,A,Annual,2016,2016,8253,NaN,NaN


In [13]:
population.shape

(2214, 17)

In [14]:
population = population[["Region", "Value"]]
population.rename(columns={"Region": "SA2pop", "Value": "Population"}, inplace=True)
population.head()

,SA2pop,Population
5,Hills District,24226
20,Skye - Sandhurst,13664
24,Lobethal - Woodside,9101
38,Berwick - South,26874
48,Central Highlands - West,8253


In [15]:
df = pd.merge(hospital_beds_by_sa2, population, left_on="SA2_NAME", right_on="SA2pop", how="left")
df.head()

,SA2_NAME,SA2_MAINCODE,Number of available beds,SA2pop,Population
0,ACT - South West,801111140,45.7,ACT - South West,5158.0
1,Acton - Upper Burnie,604011075,145.0,Acton - Upper Burnie,3206.0
2,Adelaide,401011001,812.0,Adelaide,16285.0
3,Adelaide Hills,401021003,17.0,Adelaide Hills,6914.0
4,Airlie - Whitsundays,312031359,33.0,Airlie - Whitsundays,13375.0


In [16]:
df.shape

(988, 5)

## Median Income

The Australian Bureau of Statistics provides income data for Australians by SA2 level, the source is publicly available here: https://www.abs.gov.au/statistics/labour/earnings-and-work-hours/personal-income-australia/latest-release#:~:text=Total%20personal%20income%20was%20%24903.9,87.1%25%20of%20total%20personal%20income

In [17]:
median_annual_income = pd.read_excel("Data\personal_income_Australia_SA2.xlsx", skiprows=6, sheet_name="Table 3.4")
median_annual_income = median_annual_income[["SA2 NAME", "2017-18.4"]] 
median_annual_income = median_annual_income[median_annual_income["SA2 NAME"].notnull()]

median_annual_income.head(10)

,SA2 NAME,2017-18.4
2,Braidwood,56450
3,Karabar,63385
4,Queanbeyan,63028
5,Queanbeyan - East,65101
6,Queanbeyan Region,77685
7,Queanbeyan West - Jerrabomberra,78727
8,Bombala,51435
9,Cooma,50642
10,Cooma Region,53195
11,Jindabyne - Berridale,45561


In [18]:
median_annual_income.shape

(2288, 2)

In [19]:
median_annual_income.rename(columns={"SA2 NAME": "SA2salary", "2017-18.4":"Median_Annual_Income"}, inplace=True)
median_annual_income.head()

,SA2salary,Median_Annual_Income
2,Braidwood,56450
3,Karabar,63385
4,Queanbeyan,63028
5,Queanbeyan - East,65101
6,Queanbeyan Region,77685


In [20]:
df = pd.merge(df, median_annual_income, left_on="SA2_NAME", right_on="SA2salary", how="left")
df.head()

,SA2_NAME,SA2_MAINCODE,Number of available beds,SA2pop,Population,SA2salary,Median_Annual_Income
0,ACT - South West,801111140,45.7,ACT - South West,5158.0,ACT - South West,66401
1,Acton - Upper Burnie,604011075,145.0,Acton - Upper Burnie,3206.0,Acton - Upper Burnie,46033
2,Adelaide,401011001,812.0,Adelaide,16285.0,Adelaide,54807
3,Adelaide Hills,401021003,17.0,Adelaide Hills,6914.0,Adelaide Hills,60508
4,Airlie - Whitsundays,312031359,33.0,Airlie - Whitsundays,13375.0,Airlie - Whitsundays,50559


In [21]:
df.shape

(988, 7)

## Number of Dwellings

The definition of a dwelling from the ABS is "A structure which is intended to have people live in it, and which is habitable on Census night". This can range from houses to tents. For the purpose of this dataset we will consider all structures which are habitable.

The information and source of data can be found here: https://www.abs.gov.au/ausstats/abs@.nsf/Lookup/2901.0Chapter29702016#:~:text=A%20dwelling%20is%20a%20structure,%2C%20tents%2C%20humpies%20and%20houseboats.&text=Non%2Dprivate%20dwellings%20(hotels%2C,are%20enumerated%20on%20personal%20forms.

In [22]:
no_of_dwellings = pd.read_csv(r"Data\Number_of_Dwellings.csv", encoding="utf-8")
no_of_dwellings = no_of_dwellings[(no_of_dwellings["Dwelling Type"] == "Total") & (no_of_dwellings["Dwelling Structure"] == "Total")]
no_of_dwellings.head()

,DWTD_2016,Dwelling Type,STRD_2016,Dwelling Structure,STATE,State,REGIONTYPE,Geography Level,ASGS_2016,Region,TIME,Census Year,Value,Flag Codes,Flags
128856,TOT,Total,TOT,Total,2,Victoria,SA2,Statistical Area Level 2,206041122,Melbourne,2016,2016,22429,NaN,NaN
128870,TOT,Total,TOT,Total,2,Victoria,SA2,Statistical Area Level 2,214021383,Point Nepean,2016,2016,20156,NaN,NaN
128884,TOT,Total,TOT,Total,5,Western Australia,SA2,Statistical Area Level 2,503021041,Perth City,2016,2016,19353,NaN,NaN
128898,TOT,Total,TOT,Total,1,New South Wales,SA2,Statistical Area Level 2,117031338,Waterloo - Beaconsfield,2016,2016,17003,NaN,NaN
128912,TOT,Total,TOT,Total,2,Victoria,SA2,Statistical Area Level 2,206051133,St Kilda,2016,2016,16313,NaN,NaN


In [23]:
no_of_dwellings.shape

(2301, 15)

In [24]:
no_of_dwellings = no_of_dwellings[["Region", "Value"]]
no_of_dwellings.rename(columns={"Region":"SA2Dwelling", "Value": "No_of_Dwellings"}, inplace=True)
no_of_dwellings["SA2Dwelling"] = no_of_dwellings["SA2Dwelling"].str.strip() 
no_of_dwellings.head()

,SA2Dwelling,No_of_Dwellings
128856,Melbourne,22429
128870,Point Nepean,20156
128884,Perth City,19353
128898,Waterloo - Beaconsfield,17003
128912,St Kilda,16313


In [25]:
df = pd.merge(df, no_of_dwellings, left_on="SA2_NAME", right_on="SA2Dwelling")
df.head(100)

,SA2_NAME,SA2_MAINCODE,Number of available beds,SA2pop,Population,SA2salary,Median_Annual_Income,SA2Dwelling,No_of_Dwellings
0,ACT - South West,801111140,45.7,ACT - South West,5158.0,ACT - South West,66401,ACT - South West,190
1,Acton - Upper Burnie,604011075,145.0,Acton - Upper Burnie,3206.0,Acton - Upper Burnie,46033,Acton - Upper Burnie,1628
2,Adelaide,401011001,812.0,Adelaide,16285.0,Adelaide,54807,Adelaide,8147
3,Adelaide Hills,401021003,17.0,Adelaide Hills,6914.0,Adelaide Hills,60508,Adelaide Hills,2670
4,Airlie - Whitsundays,312031359,33.0,Airlie - Whitsundays,13375.0,Airlie - Whitsundays,50559,Airlie - Whitsundays,5701
...,...,...,...,...,...,...,...,...,...
95,Boonah,310021277,489.0,Boonah,12153.0,Boonah,50374,Boonah,5051
96,Booral - River Heads,319041516,192.0,Booral - River Heads,4637.0,Booral - River Heads,51338,Booral - River Heads,1878
97,Bossley Park - Abbotsbury,127021510,271.8,Bossley Park - Abbotsbury,19498.0,Bossley Park - Abbotsbury,57331,Bossley Park - Abbotsbury,5687
98,Bouldercombe,308031206,12.0,Bouldercombe,1883.0,Bouldercombe,63399,Bouldercombe,811


In [26]:
df.shape

(988, 9)

## Employment Statistics

The Australian Bureau of Statistics provides employment data for Australians by SA2 level. There are many cuts of the data, but what will be looked at is just total employment (of any type) in an area. The source is publicly available here: https://www.abs.gov.au/statistics/labour/employment-and-unemployment/labour-force-australia/latest-release

In [27]:
employment = pd.read_csv("Data\employment_by_age_SA2.csv")
employment = employment[(employment["AGE"] == "TOT") & (employment["Sex"] == "Persons")]

In [28]:
employment.head()

,AGE,Age,SEX_ABS,Sex,INDP_C16,Industry of Employment,STATE,State,REGIONTYPE,Geography Level,ASGS_2016,Region,TIME,Census year,Value,Flag Codes,Flags
9530,TOT,Total,3,Persons,TOT,Total,1,New South Wales,SA2,Statistical Area Level 2,102011028,Avoca Beach - Copacabana,2016,2016,3630,NaN,NaN
9533,TOT,Total,3,Persons,TOT,Total,1,New South Wales,SA2,Statistical Area Level 2,102011029,Box Head - MacMasters Beach,2016,2016,4743,NaN,NaN
9536,TOT,Total,3,Persons,TOT,Total,1,New South Wales,SA2,Statistical Area Level 2,102011030,Calga - Kulnura,2016,2016,2249,NaN,NaN
9539,TOT,Total,3,Persons,TOT,Total,1,New South Wales,SA2,Statistical Area Level 2,102011031,Erina - Green Point,2016,2016,5625,NaN,NaN
9542,TOT,Total,3,Persons,TOT,Total,1,New South Wales,SA2,Statistical Area Level 2,102011032,Gosford - Springfield,2016,2016,8506,NaN,NaN


In [29]:
employment.shape

(2310, 17)

In [30]:
employment = employment[["Region", "Value"]]
employment.rename(columns={"Region": "SA2Employ", "Value":"No_of_people_earning_income"}, inplace=True)
employment.head()

,SA2Employ,No_of_people_earning_income
9530,Avoca Beach - Copacabana,3630
9533,Box Head - MacMasters Beach,4743
9536,Calga - Kulnura,2249
9539,Erina - Green Point,5625
9542,Gosford - Springfield,8506


In [31]:
df = pd.merge(df, employment, left_on="SA2_NAME", right_on="SA2Employ")
df.head()

,SA2_NAME,SA2_MAINCODE,Number of available beds,SA2pop,Population,SA2salary,Median_Annual_Income,SA2Dwelling,No_of_Dwellings,SA2Employ,No_of_people_earning_income
0,ACT - South West,801111140,45.7,ACT - South West,5158.0,ACT - South West,66401,ACT - South West,190,ACT - South West,317
1,Acton - Upper Burnie,604011075,145.0,Acton - Upper Burnie,3206.0,Acton - Upper Burnie,46033,Acton - Upper Burnie,1628,Acton - Upper Burnie,1184
2,Adelaide,401011001,812.0,Adelaide,16285.0,Adelaide,54807,Adelaide,8147,Adelaide,5972
3,Adelaide Hills,401021003,17.0,Adelaide Hills,6914.0,Adelaide Hills,60508,Adelaide Hills,2670,Adelaide Hills,3558
4,Airlie - Whitsundays,312031359,33.0,Airlie - Whitsundays,13375.0,Airlie - Whitsundays,50559,Airlie - Whitsundays,5701,Airlie - Whitsundays,6653


In [32]:
df.shape

(988, 11)

## Number of People Tertiary Educated

The Australian Bureau of Statistics provides population data with the level of education for Australians by SA2 level. There are many cuts of the data, but we will consider the total people with a bachelor's degree or higher. The source is publicly available here: https://www.abs.gov.au/statistics/people/education/education-and-work-australia/may-2020#key-statistics

In [33]:
ed_types = ["Postgraduate Degree Level", "Graduate Diploma and Graduate Certificate Level", "Bachelor Degree Level"]

terr_educated = pd.read_csv("Data\education_level_age_SA2.csv")
terr_educated = terr_educated[(terr_educated["Level of Education "].isin(ed_types)) & (terr_educated["Age"] == "Total")]
terr_educated.head()

,AGE,Age,SEX_ABS,Sex,QALLP_C16,Level of Education,STATE,State,REGIONTYPE,Geography Level,ASGS_2016,Region,TIME,Census Year,Value,Flag Codes,Flags
3,TOT,Total,3,Persons,1,Postgraduate Degree Level,1,New South Wales,SA2,Statistical Area Level 2,102011028.0,Avoca Beach - Copacabana,2016.0,2016.0,277.0,NaN,NaN
7,TOT,Total,3,Persons,2,Graduate Diploma and Graduate Certificate Level,1,New South Wales,SA2,Statistical Area Level 2,102011028.0,Avoca Beach - Copacabana,2016.0,2016.0,154.0,NaN,NaN
8,TOT,Total,3,Persons,3,Bachelor Degree Level,1,New South Wales,SA2,Statistical Area Level 2,102011028.0,Avoca Beach - Copacabana,2016.0,2016.0,1078.0,NaN,NaN
14,TOT,Total,3,Persons,1,Postgraduate Degree Level,1,New South Wales,SA2,Statistical Area Level 2,102011029.0,Box Head - MacMasters Beach,2016.0,2016.0,327.0,NaN,NaN
18,TOT,Total,3,Persons,2,Graduate Diploma and Graduate Certificate Level,1,New South Wales,SA2,Statistical Area Level 2,102011029.0,Box Head - MacMasters Beach,2016.0,2016.0,201.0,NaN,NaN


In [34]:
terr_educated = terr_educated.groupby(["Region"]).agg({"Value": ['sum']})
terr_educated.reset_index(inplace=True)
terr_educated.columns = terr_educated.columns.droplevel(-1)
terr_educated.head()

,Region,Value
0,ACT - South West,109.0
1,APY Lands,118.0
2,Abbotsford,3705.0
3,Aberfoyle Park,1919.0
4,Acacia Gardens,931.0


In [35]:
terr_educated.shape

(2310, 2)

In [36]:
terr_educated = terr_educated[["Region", "Value"]]
terr_educated.rename(columns={"Region": "SA2TEducated", "Value":"No_of_tertiary_educated"}, inplace=True)
terr_educated.head()

,SA2TEducated,No_of_tertiary_educated
0,ACT - South West,109.0
1,APY Lands,118.0
2,Abbotsford,3705.0
3,Aberfoyle Park,1919.0
4,Acacia Gardens,931.0


In [37]:
df = pd.merge(df, terr_educated, left_on="SA2_NAME", right_on="SA2TEducated", how="left")
df.head()

,SA2_NAME,SA2_MAINCODE,Number of available beds,SA2pop,Population,SA2salary,Median_Annual_Income,SA2Dwelling,No_of_Dwellings,SA2Employ,No_of_people_earning_income,SA2TEducated,No_of_tertiary_educated
0,ACT - South West,801111140,45.7,ACT - South West,5158.0,ACT - South West,66401,ACT - South West,190,ACT - South West,317,ACT - South West,109.0
1,Acton - Upper Burnie,604011075,145.0,Acton - Upper Burnie,3206.0,Acton - Upper Burnie,46033,Acton - Upper Burnie,1628,Acton - Upper Burnie,1184,Acton - Upper Burnie,135.0
2,Adelaide,401011001,812.0,Adelaide,16285.0,Adelaide,54807,Adelaide,8147,Adelaide,5972,Adelaide,5237.0
3,Adelaide Hills,401021003,17.0,Adelaide Hills,6914.0,Adelaide Hills,60508,Adelaide Hills,2670,Adelaide Hills,3558,Adelaide Hills,944.0
4,Airlie - Whitsundays,312031359,33.0,Airlie - Whitsundays,13375.0,Airlie - Whitsundays,50559,Airlie - Whitsundays,5701,Airlie - Whitsundays,6653,Airlie - Whitsundays,1359.0


In [38]:
df.shape

(988, 13)

## How many people have a mortgage

The Australian Bureau of Statistics provides population data with people who have a mortgage. Within this dataset, you can see the range of payments people make - but we will consider the total people who have any mortgage. The source is publicly available here: https://www.abs.gov.au/ausstats/abs@.nsf/mf/5609.0

In [39]:
mortgage_repay = pd.read_csv("Data\mortgage_repayments.csv")
mortgage_repay = mortgage_repay[(mortgage_repay["MRERD"] == "TOT")]
mortgage_repay = mortgage_repay[["Region", "Value"]]
mortgage_repay.rename(columns={"Region": "SA2Mortgage", "Value":"Ppl_on_Mortgage"}, inplace=True)
mortgage_repay.head()

,SA2Mortgage,Ppl_on_Mortgage
0,Goulburn,2621
1,Goulburn Region,1401
2,Yass,871
3,Yass Region,1606
4,Young,1112


In [40]:
mortgage_repay.shape

(2214, 2)

In [41]:
df = pd.merge(df, mortgage_repay, left_on="SA2_NAME", right_on="SA2Mortgage", how="left")
df.head()

,SA2_NAME,SA2_MAINCODE,Number of available beds,SA2pop,Population,SA2salary,Median_Annual_Income,SA2Dwelling,No_of_Dwellings,SA2Employ,No_of_people_earning_income,SA2TEducated,No_of_tertiary_educated,SA2Mortgage,Ppl_on_Mortgage
0,ACT - South West,801111140,45.7,ACT - South West,5158.0,ACT - South West,66401,ACT - South West,190,ACT - South West,317,ACT - South West,109.0,ACT - South West,53.0
1,Acton - Upper Burnie,604011075,145.0,Acton - Upper Burnie,3206.0,Acton - Upper Burnie,46033,Acton - Upper Burnie,1628,Acton - Upper Burnie,1184,Acton - Upper Burnie,135.0,Acton - Upper Burnie,438.0
2,Adelaide,401011001,812.0,Adelaide,16285.0,Adelaide,54807,Adelaide,8147,Adelaide,5972,Adelaide,5237.0,Adelaide,904.0
3,Adelaide Hills,401021003,17.0,Adelaide Hills,6914.0,Adelaide Hills,60508,Adelaide Hills,2670,Adelaide Hills,3558,Adelaide Hills,944.0,Adelaide Hills,1132.0
4,Airlie - Whitsundays,312031359,33.0,Airlie - Whitsundays,13375.0,Airlie - Whitsundays,50559,Airlie - Whitsundays,5701,Airlie - Whitsundays,6653,Airlie - Whitsundays,1359.0,Airlie - Whitsundays,1061.0


## How many people are renting

The Australian Bureau of Statistics provides population data with people who are renting their property. Within this dataset, you can see the range of rent payments people make - but we will consider the total people who have any mortgage. The source is publicly available here: https://www.abs.gov.au/ausstats/abs@.nsf/mf/5609.0

In [42]:
renting = pd.read_csv(r"Data\renting_SA2.csv")
renting = renting[(renting["Rent (weekly)"] == "Total")]
renting = renting[["Region", "Value"]]
renting.rename(columns={"Region": "SA2Renting", "Value":"Number_of_renters"}, inplace=True)
renting.head()

,SA2Renting,Number_of_renters
0,Avoca Beach - Copacabana,505
25,Box Head - MacMasters Beach,523
50,Calga - Kulnura,403
75,Erina - Green Point,874
100,Gosford - Springfield,3574


In [43]:
renting.shape

(2301, 2)

In [44]:
df = pd.merge(df, renting, left_on="SA2_NAME", right_on="SA2Renting", how="left")
df.head()

,SA2_NAME,SA2_MAINCODE,Number of available beds,SA2pop,Population,SA2salary,Median_Annual_Income,SA2Dwelling,No_of_Dwellings,SA2Employ,No_of_people_earning_income,SA2TEducated,No_of_tertiary_educated,SA2Mortgage,Ppl_on_Mortgage,SA2Renting,Number_of_renters
0,ACT - South West,801111140,45.7,ACT - South West,5158.0,ACT - South West,66401,ACT - South West,190,ACT - South West,317,ACT - South West,109.0,ACT - South West,53.0,ACT - South West,31
1,Acton - Upper Burnie,604011075,145.0,Acton - Upper Burnie,3206.0,Acton - Upper Burnie,46033,Acton - Upper Burnie,1628,Acton - Upper Burnie,1184,Acton - Upper Burnie,135.0,Acton - Upper Burnie,438.0,Acton - Upper Burnie,556
2,Adelaide,401011001,812.0,Adelaide,16285.0,Adelaide,54807,Adelaide,8147,Adelaide,5972,Adelaide,5237.0,Adelaide,904.0,Adelaide,3978
3,Adelaide Hills,401021003,17.0,Adelaide Hills,6914.0,Adelaide Hills,60508,Adelaide Hills,2670,Adelaide Hills,3558,Adelaide Hills,944.0,Adelaide Hills,1132.0,Adelaide Hills,227
4,Airlie - Whitsundays,312031359,33.0,Airlie - Whitsundays,13375.0,Airlie - Whitsundays,50559,Airlie - Whitsundays,5701,Airlie - Whitsundays,6653,Airlie - Whitsundays,1359.0,Airlie - Whitsundays,1061.0,Airlie - Whitsundays,1788


In [45]:
df.shape

(988, 17)

## Indigenous Population

The Australian Bureau of Statistics provides population data on the indigenous population. This additional feature to the dataset is to determine whether there is a correlation between hospital beds and high indigenous populations. The source data can be found here: https://www.abs.gov.au/statistics/people/aboriginal-and-torres-strait-islander-peoples

In [46]:
indigenous_pop = pd.read_csv(r"Data\indigenous_population_SA2.csv")
indigenous_pop = indigenous_pop[(indigenous_pop["Indigenous status"] == "Both Aboriginal and Torres Strait Islander") 
                               & (indigenous_pop["Sex"] == "Persons")]
indigenous_pop = indigenous_pop[["Region", "Value"]]
indigenous_pop.rename(columns={"Region": "SA2Indigenous", "Value":"No_of_indigenous_ppl"}, inplace=True)
indigenous_pop.head()

,SA2Indigenous,No_of_indigenous_ppl
14,Avoca Beach - Copacabana,3
32,Box Head - MacMasters Beach,3
50,Calga - Kulnura,0
68,Erina - Green Point,8
86,Gosford - Springfield,3


In [47]:
indigenous_pop.shape


(2310, 2)

In [48]:
df = pd.merge(df, indigenous_pop, left_on="SA2_NAME", right_on="SA2Indigenous", how="left")
df.head()

,SA2_NAME,SA2_MAINCODE,Number of available beds,SA2pop,Population,SA2salary,Median_Annual_Income,SA2Dwelling,No_of_Dwellings,SA2Employ,No_of_people_earning_income,SA2TEducated,No_of_tertiary_educated,SA2Mortgage,Ppl_on_Mortgage,SA2Renting,Number_of_renters,SA2Indigenous,No_of_indigenous_ppl
0,ACT - South West,801111140,45.7,ACT - South West,5158.0,ACT - South West,66401,ACT - South West,190,ACT - South West,317,ACT - South West,109.0,ACT - South West,53.0,ACT - South West,31,ACT - South West,0
1,Acton - Upper Burnie,604011075,145.0,Acton - Upper Burnie,3206.0,Acton - Upper Burnie,46033,Acton - Upper Burnie,1628,Acton - Upper Burnie,1184,Acton - Upper Burnie,135.0,Acton - Upper Burnie,438.0,Acton - Upper Burnie,556,Acton - Upper Burnie,5
2,Adelaide,401011001,812.0,Adelaide,16285.0,Adelaide,54807,Adelaide,8147,Adelaide,5972,Adelaide,5237.0,Adelaide,904.0,Adelaide,3978,Adelaide,7
3,Adelaide Hills,401021003,17.0,Adelaide Hills,6914.0,Adelaide Hills,60508,Adelaide Hills,2670,Adelaide Hills,3558,Adelaide Hills,944.0,Adelaide Hills,1132.0,Adelaide Hills,227,Adelaide Hills,0
4,Airlie - Whitsundays,312031359,33.0,Airlie - Whitsundays,13375.0,Airlie - Whitsundays,50559,Airlie - Whitsundays,5701,Airlie - Whitsundays,6653,Airlie - Whitsundays,1359.0,Airlie - Whitsundays,1061.0,Airlie - Whitsundays,1788,Airlie - Whitsundays,22


In [49]:
df.shape

(988, 19)

In [50]:
df.drop(["SA2pop", "SA2salary", "SA2Dwelling", "SA2Employ", "SA2TEducated", "SA2Mortgage", "SA2Renting",
        "SA2Indigenous"], axis=1, inplace=True)
df.head()

,SA2_NAME,SA2_MAINCODE,Number of available beds,Population,Median_Annual_Income,No_of_Dwellings,No_of_people_earning_income,No_of_tertiary_educated,Ppl_on_Mortgage,Number_of_renters,No_of_indigenous_ppl
0,ACT - South West,801111140,45.7,5158.0,66401,190,317,109.0,53.0,31,0
1,Acton - Upper Burnie,604011075,145.0,3206.0,46033,1628,1184,135.0,438.0,556,5
2,Adelaide,401011001,812.0,16285.0,54807,8147,5972,5237.0,904.0,3978,7
3,Adelaide Hills,401021003,17.0,6914.0,60508,2670,3558,944.0,1132.0,227,0
4,Airlie - Whitsundays,312031359,33.0,13375.0,50559,5701,6653,1359.0,1061.0,1788,22


In [51]:
# df["No_of_Dwellings"] = df["No_of_Dwellings"].str.strip()
df["No_of_Dwellings"] = pd.to_numeric(df['No_of_Dwellings'],errors='coerce')
df["Median_Annual_Income"] = pd.to_numeric(df['Median_Annual_Income'],errors='coerce')
df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 988 entries, 0 to 987
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   SA2_NAME                     988 non-null    object 
 1   SA2_MAINCODE                 988 non-null    int64  
 2   Number of available beds     988 non-null    float64
 3   Population                   943 non-null    float64
 4   Median_Annual_Income         979 non-null    float64
 5   No_of_Dwellings              982 non-null    float64
 6   No_of_people_earning_income  988 non-null    int64  
 7   No_of_tertiary_educated      988 non-null    float64
 8   Ppl_on_Mortgage              943 non-null    float64
 9   Number_of_renters            988 non-null    int64  
 10  No_of_indigenous_ppl         988 non-null    int64  
dtypes: float64(6), int64(4), object(1)
memory usage: 92.6+ KB


## Output for EDA work

This concludes this workbook as it's specifically designed to work as an investigative data pipe. 

In [52]:
df.to_excel(r"Output_data\main_dataset.xlsx", index=False, na_rep=0)

In [53]:
cleaned_df = pd.read_excel("Output_data\main_dataset.xlsx")
cleaned_df.head()

,SA2_NAME,SA2_MAINCODE,Number of available beds,Population,Median_Annual_Income,No_of_Dwellings,No_of_people_earning_income,No_of_tertiary_educated,Ppl_on_Mortgage,Number_of_renters,No_of_indigenous_ppl
0,ACT - South West,801111140,45.7,5158,66401,190,317,109,53,31,0
1,Acton - Upper Burnie,604011075,145.0,3206,46033,1628,1184,135,438,556,5
2,Adelaide,401011001,812.0,16285,54807,8147,5972,5237,904,3978,7
3,Adelaide Hills,401021003,17.0,6914,60508,2670,3558,944,1132,227,0
4,Airlie - Whitsundays,312031359,33.0,13375,50559,5701,6653,1359,1061,1788,22
